In [1]:


# ________________Manipulation_____________
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from itertools import product
import matplotlib.pyplot as plt
import joblib as jl
import os

#___________________ACF___________________
from statsmodels.tsa.stattools import acf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Importing Data

In [2]:
#__________________Data Retrieval____________________

# Local version of the same logic
DATASET_PATH = "C:/Users/LENOVO/Desktop/Sales Forecasing/data/raw"

print("Listing files in:", DATASET_PATH)
for dirname, _, filenames in os.walk(DATASET_PATH):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Listing files in: C:/Users/LENOVO/Desktop/Sales Forecasing/data/raw
C:/Users/LENOVO/Desktop/Sales Forecasing/data/raw\items.csv
C:/Users/LENOVO/Desktop/Sales Forecasing/data/raw\item_categories.csv
C:/Users/LENOVO/Desktop/Sales Forecasing/data/raw\sales_train.csv
C:/Users/LENOVO/Desktop/Sales Forecasing/data/raw\sample_submission.csv
C:/Users/LENOVO/Desktop/Sales Forecasing/data/raw\shops.csv
C:/Users/LENOVO/Desktop/Sales Forecasing/data/raw\test.csv


In [3]:
# Importing  the different dataframes
df_sales_train = pd.read_csv(
    "C:/Users/LENOVO/Desktop/Sales Forecasing/data/raw/sales_train.csv",
    parse_dates=["date"],
)
df_sales_test = pd.read_csv(
    "C:/Users/LENOVO/Desktop/Sales Forecasing/data/raw/test.csv"
)
df_sample_submission = pd.read_csv(
    "C:/Users/LENOVO/Desktop/Sales Forecasing/data/raw/sample_submission.csv"
)
df_item_info = pd.read_csv(
    "C:/Users/LENOVO/Desktop/Sales Forecasing/data/raw/items.csv"
)
df_item_cat = pd.read_csv(
    "C:/Users/LENOVO/Desktop/Sales Forecasing/data/raw/item_categories.csv"
)
df_shop_info = pd.read_csv(
    "C:/Users/LENOVO/Desktop/Sales Forecasing/data/raw/shops.csv"
)

In [4]:
df_sales_test.head(3)

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233


In [5]:
print(df_sales_train.shape)
df_sales_train.head(3)

(2935849, 6)


,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.0,1.0
1,03.01.2013,0,25,2552,899.0,1.0
2,05.01.2013,0,25,2552,899.0,-1.0


In [6]:
print(df_item_info.shape)
df_item_info.head(3)

(22170, 3)


,item_name,item_id,category_id
0,!! IN THE POWER OF HAPPINESS (PLAST) D,0,40
1,! ABBYY FineReader 12 Professional Edition Ful...,1,76
2,*** IN THE GLORY OF THE GLORY (UNV) D,2,40


In [7]:
print(df_item_cat.shape)
df_item_cat.head(3)

(84, 2)


,item_category_name,item_category_id
0,PC - Headsets / Headphones,0
1,Accessories - PS2,1
2,Accessories - PS3,2


In [8]:
print(df_shop_info.shape)
df_shop_info.head(3)

(60, 2)


,shop_name,shop_id
0,"! Yakutsk Ordzhonikidze, 56 francs",0
1,"! Yakutsk TC ""Central"" fran",1
2,"Adygea TC ""Mega""",2


# Join all Dataframes

In [9]:
# Creating the Full Dataset
print("The Full Dataframe is the following")
df_sales_train1 = df_sales_train.join(
    other=df_shop_info, on="shop_id", how="inner", rsuffix="_"
).drop(
    columns="shop_id_"
)  # we add rsuffix, because when it's not like merge, here it adds the shop_id twice,and if we don't add the suffix it we will have 2 columns wit the same
df_sales_train1 = df_sales_train1.join(
    other=df_item_info, on="item_id", how="inner", rsuffix="_"
).drop(columns="item_id_")

df_sales_train1 = df_sales_train1.rename(
    columns={"category_id": "item_category_id"}
)
df_sales_train1 = df_sales_train1.join(
    other=df_item_cat, on="item_category_id", how="inner", rsuffix="_"
).drop(columns="item_category_id_")
df_sales_train1

The Full Dataframe is the following


,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,shop_name,item_name,item_category_id,item_category_name
0,02.01.2013,0,59,22154,999.0,1.0,"Yaroslavl TC"" Altair """,SCENE 2012 (BD),37,Cinema - Blu-Ray
25918,02.01.2013,0,25,22154,999.0,1.0,"Moscow TEC"" Atrium """,SCENE 2012 (BD),37,Cinema - Blu-Ray
25919,03.01.2013,0,25,22154,999.0,1.0,"Moscow TEC"" Atrium """,SCENE 2012 (BD),37,Cinema - Blu-Ray
25920,20.01.2013,0,25,22154,999.0,1.0,"Moscow TEC"" Atrium """,SCENE 2012 (BD),37,Cinema - Blu-Ray
25921,23.01.2013,0,25,22154,999.0,1.0,"Moscow TEC"" Atrium """,SCENE 2012 (BD),37,Cinema - Blu-Ray
...,...,...,...,...,...,...,...,...,...,...
2896869,22.10.2015,33,55,13093,250.0,1.0,Digital warehouse 1C-Online,Windows payment card: 250 rubles [Digital vers...,36,Payment cards - Windows (Digital)
2851121,21.09.2015,32,55,13091,1000.0,1.0,Digital warehouse 1C-Online,Windows payment card: 1000 rubles [Digital ver...,36,Payment cards - Windows (Digital)
2851125,16.09.2015,32,55,13094,2500.0,1.0,Digital warehouse 1C-Online,Windows payment card: 2500 rubles [Digital ver...,36,Payment cards - Windows (Digital)
2851126,22.09.2015,32,55,13094,2500.0,2.0,Digital warehouse 1C-Online,Windows payment card: 2500 rubles [Digital ver...,36,Payment cards - Windows (Digital)


In [10]:
print(len(df_sales_train1[df_sales_train1['item_cnt_day']<0]))
df_sales_train1[df_sales_train1['item_cnt_day']<0]

7356


,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,shop_name,item_name,item_category_id,item_category_name
2628513,19.05.2015,28,31,8095,499.00,-1.0,"Moscow TC"" Semenovsky """,AVATAR (BD + DVD),37,Cinema - Blu-Ray
194532,17.02.2013,1,44,8095,499.00,-1.0,Samara Melody shopping center,AVATAR (BD + DVD),37,Cinema - Blu-Ray
2587625,01.04.2015,27,44,8095,499.00,-1.0,Samara Melody shopping center,AVATAR (BD + DVD),37,Cinema - Blu-Ray
1795950,25.07.2014,18,45,8095,499.00,-1.0,Samara ParkHouse shopping center,AVATAR (BD + DVD),37,Cinema - Blu-Ray
1862738,19.07.2014,18,25,8266,199.34,-1.0,"Moscow TEC"" Atrium """,AMERICAN PIE. ALL IN ASSEMBLY (BD),37,Cinema - Blu-Ray
...,...,...,...,...,...,...,...,...,...,...
1927139,12.08.2014,19,12,11367,250.00,-1.0,Internet-shop of emergency situations,Delivery (Cleo-Logistic),9,Delivery of goods
1927140,07.08.2014,19,12,11367,375.00,-2.0,Internet-shop of emergency situations,Delivery (Cleo-Logistic),9,Delivery of goods
1927141,04.08.2014,19,12,11367,250.00,-2.0,Internet-shop of emergency situations,Delivery (Cleo-Logistic),9,Delivery of goods
1487597,14.03.2014,14,55,1982,599.00,-1.0,Digital warehouse 1C-Online,"Borderlands 2 (SoftKlab) [PC, Digital Version]",31,PC Games - Digital


There are arround 7000 rows where the item quantity is negative, we will assume these are returns and drop them from the df

In [11]:
df_sales_train1[df_sales_train1['item_price']<0]

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,shop_name,item_name,item_category_id,item_category_name
484683,15.05.2013,4,32,2973,-1.0,1.0,"Moscow TC"" Serebryany House ""","DmC Devil May Cry [PS3, Russian subtitles]",19,Games - PS3


In [12]:
# Deleting the Returns and the negative amounts 
df=df_sales_train1.copy()
df_sales_train1=df_sales_train1[(df_sales_train1['item_price']>0) & (df_sales_train1['item_cnt_day']>0)]
df_sales_train1

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,shop_name,item_name,item_category_id,item_category_name
0,02.01.2013,0,59,22154,999.0,1.0,"Yaroslavl TC"" Altair """,SCENE 2012 (BD),37,Cinema - Blu-Ray
25918,02.01.2013,0,25,22154,999.0,1.0,"Moscow TEC"" Atrium """,SCENE 2012 (BD),37,Cinema - Blu-Ray
25919,03.01.2013,0,25,22154,999.0,1.0,"Moscow TEC"" Atrium """,SCENE 2012 (BD),37,Cinema - Blu-Ray
25920,20.01.2013,0,25,22154,999.0,1.0,"Moscow TEC"" Atrium """,SCENE 2012 (BD),37,Cinema - Blu-Ray
25921,23.01.2013,0,25,22154,999.0,1.0,"Moscow TEC"" Atrium """,SCENE 2012 (BD),37,Cinema - Blu-Ray
...,...,...,...,...,...,...,...,...,...,...
2896869,22.10.2015,33,55,13093,250.0,1.0,Digital warehouse 1C-Online,Windows payment card: 250 rubles [Digital vers...,36,Payment cards - Windows (Digital)
2851121,21.09.2015,32,55,13091,1000.0,1.0,Digital warehouse 1C-Online,Windows payment card: 1000 rubles [Digital ver...,36,Payment cards - Windows (Digital)
2851125,16.09.2015,32,55,13094,2500.0,1.0,Digital warehouse 1C-Online,Windows payment card: 2500 rubles [Digital ver...,36,Payment cards - Windows (Digital)
2851126,22.09.2015,32,55,13094,2500.0,2.0,Digital warehouse 1C-Online,Windows payment card: 2500 rubles [Digital ver...,36,Payment cards - Windows (Digital)


In [13]:
df_sales_train1.isnull().sum()

date                  0
date_block_num        0
shop_id               0
item_id               0
item_price            0
item_cnt_day          0
shop_name             0
item_name             0
item_category_id      0
item_category_name    0
dtype: int64

In [15]:
df_c=df_sales_train1.copy()
df_sales_train1["date"] = pd.to_datetime(
    df_c["date"], format="%d.%m.%Y", dayfirst=True
)
df_sales_train1["week_id"] = df_sales_train1["date"].dt.strftime("%Y-W%U")
df_sales_train1["Revenue"] = (
    df_sales_train1["item_price"] * df_sales_train1["item_cnt_day"]
)
df_sales_train1.head(3)

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_6792\2929603309.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sales_train1["date"] = pd.to_datetime(
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_6792\2929603309.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sales_train1["week_id"] = df_sales_train1["date"].dt.strftime("%Y-W%U")
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_6792\2929603309.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using 

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,shop_name,item_name,item_category_id,item_category_name,week_id,Revenue
0,2013-01-02,0,59,22154,999.0,1.0,"Yaroslavl TC"" Altair """,SCENE 2012 (BD),37,Cinema - Blu-Ray,2013-W00,999.0
25918,2013-01-02,0,25,22154,999.0,1.0,"Moscow TEC"" Atrium """,SCENE 2012 (BD),37,Cinema - Blu-Ray,2013-W00,999.0
25919,2013-01-03,0,25,22154,999.0,1.0,"Moscow TEC"" Atrium """,SCENE 2012 (BD),37,Cinema - Blu-Ray,2013-W00,999.0


In [16]:
folder_path = "C:/Users/LENOVO/Desktop/Sales Forecasing/data/processed"
file_path= os.path.join(folder_path, "df_sales_train1.pkl")
# Save the df to  a pikle file
os.makedirs(folder_path,exist_ok=True)
jl.dump(df_sales_train1, file_path)

['C:/Users/LENOVO/Desktop/Sales Forecasing/data/processed\\df_sales_train1.pkl']